In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# --- PREPROCESAMIENTO Y ENTRENAMIENTO ----------------------------
# 1. Convertir las variables categóricas a numéricas (one-hot)
cat_cols = ['sex', 'smoker', 'region']
dataset = pd.get_dummies(dataset, columns=cat_cols, drop_first=True)

# 2. Separar etiquetas y características
labels = dataset.pop('expenses')          # ← y = gastos
features = dataset                         # ← X

# 3. Dividir en entrenamiento (80 %) y prueba (20 %)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.20, random_state=42)

# 4. Escalar las columnas numéricas para acelerar el gradiente
from sklearn.preprocessing import StandardScaler
num_cols   = ['age', 'bmi', 'children']
scaler     = StandardScaler()

X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])

# 5. Convertir a tensores
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test  = tf.convert_to_tensor(X_test,  dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test  = tf.convert_to_tensor(y_test,  dtype=tf.float32)

# 6. Definir el modelo
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dense(64,  activation='relu'),
    layers.Dense(32,  activation='relu'),
    layers.Dense(1)                                   # salida continua
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',                   # error cuadrático medio
    metrics=['mae', 'mse']        # mostramos MAE y MSE
)

# 7. Entrenar con early stopping para evitar sobre-ajuste
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_mae', patience=30, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,         # 20 % de entrenamiento se usa como validación
    epochs=500,
    batch_size=32,
    callbacks=[early_stop],
    verbose=0                     # silencia salida; pon 1 si quieres ver el log
)

# 8. Renombrar variables como las espera la celda de prueba
train_dataset, test_dataset = X_train, X_test
train_labels,  test_labels  = y_train, y_test
# -----------------------------------------------------------------


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
